# Pet Grooming Machine Learning Project

By JP Mendieta

# 1. Background

My parents own a small pet grooming.  Over the years, we've incorporated more and more technology into the business, from setting up credit card terminals, to storing client information in a client management application.  We've never done any analytics on our clientele though. We have an idea of when our peak business months are, but we don't have any metrics to back it up.  Additionally,  every client has their own grooming schedule.  Some bring their pets every month, others once a year.  It would be really useful to get a better understanding of how often a client brings their pet and what factors influence their grooming schedule.  Knowing this will help us, in advance, prepare our inventory (how much shampoo should be ordered, the type of shampoo) and also shape our outreach initiatives.  Ideally, we would like to target and retain clients who are likely to bring their pets more frequently.

# 2. Problem Statement

**Can I predict how often a customer will bring his/her pet(s) to be groomed?**

My initial hypotheses:
1. The breed of the dog is a good predictor of grooming frequency.  This could probably be broken further down into long-haired vs short-haired. For example, long-haired dogs require more maintenance.
2. The number of pets a client has is also a good predictor.  The more pets a client has, the more costly the grooming will be.  The high cost could deter clients from bringing their pets more frequently.
3. The client's commute is also a good predictor. The longer the commute, the less likely the client will make the trip.

# 3. Data

## 3.1 Client Management Application

We've been using a client management application for a number of years now.  This was a huge upgrade over our previous client management system - several shoeboxes full of 4x6 index cards.  Nowadays, any time a new client comes to the grooming, the receptionist simply inputs the client's data directly to the app.  Similarly, if a client gets a new pet, the receptionist can easily update that client's profile. The UI is well-designed and incredibly easy to navigate.  Even my mom can use it.

The application stores the data on a Microsoft SQL Server.  After doing some exploring, I found that the app provides an option to export the tables as tab-delimited txt files.  The two main tables that get populated are the **Clients** table and the **Pets** table.

Because the tables were saved as tab-delimited txt files, it was simple to load them to pandas for analysis.

`clients = pd.read_table('Clients.txt', sep='\t')`

*A small representation of the client data:*

Address|City|Phone|FirstName|Zipcode
---|---|---|---|---
100 Apple St.|Olney|222-555-1234|Joe|20833
210 Pear Dr.|Rockville|301-555-1234|Anna|20832
3010 Strawberry Ln.|Reston|240-535-1255|Sam|20171

`clients.shape` yields: (3189, 55)

Quickly, we can see there are over 3,000 clients.  Though there are 55 fields, many of them are null or empty. For many of our clients, asking them for their **Date of Birth** would probably result in us getting the evil eye.

*A small representation of the pet data:*

ClientID|Breed|Gender|DOB|Barker|Name
---|---|---|---|---|---
4750|Pug|M|10/01/2008|F|Pugsley
4750|Golden Retriever|F|05/05/2013|F|Goldie
4752|Shih Tzu|F|07/01/2014|F|Sue

`pets.shape` yields: (3989,40)

Some of fields are populated with default values (i.e. F, or False, for Barker), which was initially misleading when doing a `pets.isnull().sum()` count.

The **Pets** table is connected to the **Clients** table through the **ClientID** foreign key.  Doing a count of the pet observations by ClientID returns the number of pets each client has:

`num_pets = pets.ClientID.value_counts().head(5)`

ClientID|Count
---|---
4675|9
5644|6
4384|5
5107|5
5298|5

Reviewing the counts brought up some more questions, such as, did Client 4675 really have 9 pets?  Maybe the client had six pets, three passed away, so he/she then got three more.

I ran several more counts (only the top 5 rows of each are shown).
* `cities = clients.City.value_counts()`

City|Count
---|---
Olney|949
Silver Spring|551
Brookeville|472
Rockville|323
Gaithersburg|179
* `zipcodes = clients.ZIP.value_counts()`

ZIP|Count
---|---
20832|932
20833|470
20853|247
20906|216
20905|177
* `pet_breeds = pets.Breed.value_counts()`

Breed|Count
---|---
Shih Tzu|279
Goldern Retriever|219
Bichon Frise|161
Yorkshire Terrier|155
Maltese|148

Reviewing the highest city counts, I could already see that the farther the city, the less clients we have.  There are in total 485 different breeds too.  This might not make a great feature due to it's high variety, but low counts (there are several dozen mixed breeds that only have a count of 1).

## 3.2 Accounting Spreadsheets

The second data source I expected to use was the appointment data.  Unfortunately, after talking to my parents and the receptionists, I found out the appointments are still maintained in an appointment notebook. I did some more exploring and found accounting spreadsheets for every day of the past four years.  These spreadsheets have the following format:
* Title: **April 1, 2015.xlsx**

Credit||Check||Cash||Tip||
---|---|---|---|---|---|---|---
Faber|135|Mendes|145|Dillashaw|135|Grillo|20
Aldo|45|Mcgregor|145|Edgar|75|Jones|15
Jones|205|||Cormier|90||
|||||||
|||||||
Total|385|Total|380|Total|210|Total|35
|||||||
|||Day Total|930|||

I realized if I counted all of the names in the spreadsheet, I would at least know how many clients came that day.  I then wrote a script to do a count of all of the names for all of the spreadsheets for the past four years:

```python
date_client_count = []
directory = '/Users/jp/Projects/janneths/data/Payments/'
for path, dirs, files in os.walk(directory):
        for f in files:     
            if f[0] not in ['.','~']:
                #Zero out counts
                credit_count = 0
                check_count = 0
                cash_count = 0
                
                #Read single spreadsheet into dataframe
                payment = pd.read_excel(os.path.join(path,f),sheetname=0)
                
                #Find counts
                credit_count = payment.Credit[payment.Credit.notnull() & (payment.Credit != 'Total')].describe()['count']
                check_count = payment.Check[payment.Check.notnull() & (payment.Check != 'Total') & (payment.Check != 'Day Total')].describe()['count']
                cash_count = payment.Cash[payment.Cash.notnull() & (payment.Cash != 'Total')].describe()['count']
                
                total_count = credit_count + check_count + cash_count
                
                #Convert filename to datetime object
                date = datetime.datetime.strptime(f.replace('.xlsx',''), '%B %d, %Y')
                day_of_week = date.weekday()
                
                #Store results in master list
                date_client_count.append((date,day_of_week,credit_count,check_count,cash_count, total_count))

#Load counts to a dataframe
df_dates = pd.DataFrame(date_client_count, columns=['Date','Weekday','Credit_Count','Check_Count','Cash_Count','Total_Count'])
df_dates.sort('Date')
```

In addition to collecting the counts, I also converted the filenames into datetime values.  I later realized I could probably have done this directly in the dataframe. Having the datetime values allowed me to plot the counts over time.  I plotted the counts for the year 2015:

```python
df_2015 = df_dates[(df_dates.Date >= '2015-01-01') & (df_dates.Date < '2016-01-01')]
df_2015.plot(kind='line', x='Date', y='Total_Count', title = 'Client Counts')
```

![alt text](../images/client_counts_2015.png)

Reviewing the plot, it looks like client counts increase as the weather gets warmer, then declines again as the weather cools off.  I then plotted the counts for a specific month:

![alt text](../images/client_counts_aug2015.png)

There appears to be clear peaks and valleys.  I calculated the mean counts by day of the week for this month and got the following:

Weekday|Mean Count
---|---
0|22.6
1|27.5
2|20
3|23.75
4|26.5
5|29

Wednesday (Weekday 2) has the lowest mean count, whereas Saturday (Weekday 5) has the highest mean count.

# 4. Feature Analysis

### 4.1 Breed

The first feature I specified in the problem statement is **breed**. There are 485 unique breeds, but there are not enough clients for each breed to effectively model the data against.  Breed is still a valuable launching pad to engineer other potential significant features.  I previously mentioned long-haired vs short-haired, but this trait might be hard to collect for each breed.  Alternatively,  I could categorize the pets by **size**.  If a dog is equal to or less than 30lbs, I can classify it as **small**.  If it is greater than 20lbs, I can classify it as **large**.  Instinctively, I can make the assumption that large dogs tend to be outside more often than small dogs.  I found a website (http://modernpuppies.com/breedweightchart.aspx) that lists average weights by dog breed for over 171 breeds:

Breed|Average Adult Weight|Average Adult Weight
---|---|---
Affenpinscher|Male:7-9lbs|Female:7-9lbs
Afghan Hound|Male:60lbs|Female:50lbs
Akita|Male:85-130lbs|Female:65-110lbs

I copy and pasted the chart to a spreadsheet, but I will still need to write a script to parse and map the weights to my **Pets** dataframe.  I might have to also adjust my weight threshold or introduce a third, **medium size**, category.

### 4.2 Number of Pets 

The number of pets per client is already in the data. As shown in **Section 3.1**, doing a count of the pet observations, in the **Pet** table, by **ClientID** returns the number of pets each client has.

### 4.3 Commute

Reviewing the client counts by city,  I could clearly see a drop in clients the further the city was from the grooming. There are 90 unique cities in the data.  I could use the Google Maps API (https://developers.google.com/maps/documentation/geocoding/intro) to geocode the addresses and then calculate the distance of each client lat-long coordinate to the grooming lat-long.

### 4.4 Other Features

I asked my parents what factors they thought influenced clients to bring their pets more frequently.  My dad mentioned that clients without kids are more likely to bring their pets more frequently because they put more attention to their pets.  The receptionist thought it was more of a socio-economic factor. I don't have access to this data about our clientele though.  I could use census data, but a lot of it is at the county level, which is too broad for my needs.

### 4.5 Response Variable

Despite all the digging and data exploitation I found myself doing, I still could not collect the response variable I needed - how frequently a client comes to the grooming.  The **accounting spreadsheets** contain the clients' last names.  An option would be to map the counts by client last name (i.e. Mendieta came to the grooming five times in 2015).  This would be inaccurate though, since clients with the same last name would be grouped together.  A possible solution is to do a count of client last names and only use clients with unique last names.  This would reduce the size of the dataset, but I would have to check if the difference is significant.  My other option is to give up and just modify my problem statement.

# 5. Modeling

With this problem statement still in mind, I would first use a K-Nearest Neighbor Classification model. The response variable would be a binary response: frequent vs not frequent. Plotting the frequency counts by client will provide a quick visualization on if there is a clear separation in frequency counts; below that threshold can be considered not frequent and above frequent.

# 6. Conslusion

Clearly, I still have some work to do.  Some positives from this project are that the receptionists are now inputting appointments into the client management application.  I would still like to pursue this problem statement, but I am also brainstorming ways to still use the data I've collected, but on a slightly different statement, such as: Can I predict what type of dog a client has?